In [1]:
import illustris_python as il
import snapshot as sn
import numpy as np
import h5py

Let us take the mass range as between $10^{11} M_\odot$ ans $10^{15} M_\odot$, load their particles and compute their masses.

In [2]:
h = 0.6774
z = 0
a = 1 / (1 + z)
max_mass = 1e15
min_mass = 1e11

In [3]:
basePath = '../sims.TNG/TNG100-1/output/'

In [4]:
header = il.groupcat.loadHeader(basePath, 99)

In [5]:
header['Ngroups_Total']

6291349

In [6]:
halos = il.groupcat.loadHalos(basePath, 99, fields='GroupMass')

In [7]:
np.argmax(halos), np.argmin(halos)

(0, 6038642)

In [8]:
np.min(halos), halos[-1], halos[6038642]

(0.0024157632, 0.0031525954, 0.0024157632)

In [9]:
halos_mass = halos * 1e10 / h

In [10]:
req_indices = np.where((halos_mass > min_mass) & (halos_mass < max_mass))[0]

In [11]:
len(req_indices) 

17292

First, let us compute the total mass of the shortlisted halos as given in the group catalog

In [12]:
total_sum = 0
for i in range(len(req_indices)):
    total_sum += halos_mass[req_indices[i]]

In [13]:
total_sum # Summing the pre-existing halo masses given in the Group Catalog

2.082994693993267e+16

Now, let us find the sum of individual masses of different particles in the shortlisted halos

In [14]:
part_type = [0, 4, 5]

In [15]:
part_data = np.zeros((len(req_indices), len(part_type)))

In [16]:
fields_h = ['Masses']

In [17]:
check = 0
for i in range(len(req_indices)):
    count = 0
    for j in range(len(part_type)):
        data = sn.loadHalo(basePath, 99, req_indices[i], part_type[j], fields=fields_h)
        
        # Check if the loaded data is a dictionary
        if isinstance(data, dict):
            part_data[i, j] = 0  # Skip summing if it's a dictionary (i.e., if count is 0)
        else:
            part_data[i, j] = np.sum(data)
            
        count += 1
    check += 1

In [18]:
check, count

(17292, 3)

In [19]:
bary_sum = np.zeros(len(part_type))

In [20]:
for i in range(len(part_type)):
    bary_sum[i] = np.sum(part_data[:, i]) * 1e10 / h

In [21]:
bary_sum

array([1.83939016e+15, 4.29755293e+14, 1.58756697e+12])

In [22]:
total_sum1 = np.sum(bary_sum)

In [23]:
total_sum1 # Computed sum of masses of all the baryonic particles

2270733015987133.0

In [24]:
total_sum / total_sum1

9.17322591131546

Since, the total mass is one order of magnitude larger than the sum of the baryonic masses, we need to consider the masses of the dark matter particles also.

In [25]:
snapshot_path = sn.snapPath(basePath, 99)

In [26]:
hdf = h5py.File(snapshot_path, 'r')

In [27]:
masses = hdf['Header'].attrs['MassTable']

In [28]:
mass_dm = masses[1] * 1e10 / h

In [29]:
halo_info = il.groupcat.loadHalos(basePath, 99, fields='GroupLenType')

In [30]:
np.shape(halo_info)

(6291349, 6)

In [31]:
total_dm = np.sum(halo_info[req_indices, 1])

In [32]:
total_dm # Total number of dm particles

2486679292

In [33]:
total_dm_mass = (mass_dm * total_dm) # Total mass of dm particles

In [34]:
sum_total = np.append(bary_sum, total_dm_mass)

In [35]:
sum_total # Total sum of masses of all particles

array([1.83939016e+15, 4.29755293e+14, 1.58756697e+12, 1.85592137e+16])

Finally, let us load the pre-existing part-wise masses from the group catalog

In [36]:
parts = il.groupcat.loadHalos(basePath, 99, fields='GroupMassType')

In [37]:
part_0 = np.zeros(len(req_indices))
part_1 = np.zeros(len(req_indices))
part_4 = np.zeros(len(req_indices))
part_5 = np.zeros(len(req_indices))

In [38]:
for i in range(len(req_indices)):
    part_0[i] = parts[req_indices[i], 0] * 1e10 / h
    part_1[i] = parts[req_indices[i], 1] * 1e10 / h
    part_4[i] = parts[req_indices[i], 4] * 1e10 / h
    part_5[i] = parts[req_indices[i], 5] * 1e10 / h

In [39]:
part_total = np.array([np.sum(part_0), np.sum(part_1), np.sum(part_4), np.sum(part_5)])

In [40]:
part_total # Summing the pre-existing part-wise masses of all the particles

array([1.83994736e+15, 1.85592136e+16, 4.29198195e+14, 1.58756697e+12])

In [41]:
total_sum, np.sum(part_total), np.sum(sum_total)

(2.082994693993267e+16, 2.0829946746544244e+16, 2.082994667877565e+16)

In [42]:
total_sum / np.sum(sum_total)

1.0000000125375752

In [43]:
np.sum(part_total) / np.sum(sum_total)

1.0000000032534215

In [44]:
total_sum / np.sum(part_total)

1.0000000092841537

Hence, it clear that total_sum ~ np.sum(part_total) ~ np.sum(sum_total).